<a href="https://colab.research.google.com/github/cs236299-2023-spring/lab3-3-HayElmaliah/blob/master/lab3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Please do not change this cell because some hidden tests might depend on it.
import os

# Otter grader does not handle ! commands well, so we define and use our
# own function to execute shell commands.
def shell(commands, warn=True):
    """Executes the string `commands` as a sequence of shell commands.

       Prints the result to stdout and returns the exit status.
       Provides a printed warning on non-zero exit status unless `warn`
       flag is unset.
    """
    file = os.popen(commands)
    print (file.read().rstrip('\n'))
    exit_status = file.close()
    if warn and exit_status != None:
        print(f"Completed with errors. Exit status: {exit_status}\n")
    return exit_status

shell("""
ls requirements.txt >/dev/null 2>&1
if [ ! $? = 0 ]; then
 rm -rf .tmp
 git clone https://github.com/cs236299-2023-spring/lab3-3.git .tmp
 mv .tmp/tests ./
 mv .tmp/requirements.txt ./
 rm -rf .tmp
fi
pip install -q -r requirements.txt
""")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.0/164.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.9 MB/s eta 0:00:00


In [2]:
# Initialize Otter
import otter
grader = otter.Notebook()

$$
\renewcommand{\vect}[1]{\mathbf{#1}}
\renewcommand{\cnt}[1]{\sharp(#1)}
\renewcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\renewcommand{\softmax}{\operatorname{softmax}}
\renewcommand{\Prob}{\Pr}
\renewcommand{\given}{\,|\,}
$$

# Course 236299
## Lab 3-3 - Probabilistic context-free grammars

In previous labs, you have practiced constituency parsing using context-free grammars with the CKY parsing algorithm. In this lab you will extend this framework to a probabilistic one, probabilistic context-free grammars (PCFG).

New bits of Python used for the first time in the _solution set_ for this lab, and which you may therefore find useful:

* [`math.prod`](https://docs.python.org/3/library/math.html#math.prod)
* [`nltk.tree.Tree.productions`](https://www.nltk.org/api/nltk.html?highlight=production#nltk.tree.Tree.productions)

# Preparations

In [3]:
import math
import nltk
import operator

from collections import Counter, defaultdict
from pprint import pprint

# Syntactic ambiguity

Let's start with the following simplified grammar for arithmetic word expressions from the last lab:

In [4]:
arithmetic_grammar = nltk.CFG.fromstring("""
    S -> NUM | S OP S
    OP -> ADD | MULT

    NUM -> 'zero' | 'one' | 'two' | 'three' | 'four' | 'five'
    NUM -> 'six' | 'seven' | 'eight' | 'nine' | 'ten'

    ADD -> 'plus'
    MULT -> 'times'
""")

As a running example throughout this lab, we'll use the example phrase "two times three plus four".

In [5]:
example = "two plus three times four"

We can use the given CFG to parse this example phrase and print the possible parse trees.

In [6]:
parser = nltk.parse.BottomUpChartParser(arithmetic_grammar)
parses = list(parser.parse(example.split()))

for i, tree in enumerate(parses):
  print(f"Parse {i+1}:\n")
  tree.pretty_print()

Parse 1:

           S             
      _____|__________    
     S           |    |  
  ___|_____      |    |   
 S   OP    S     OP   S  
 |   |     |     |    |   
NUM ADD   NUM   MULT NUM 
 |   |     |     |    |   
two plus three times four

Parse 2:

           S             
  _________|_____         
 |   |           S       
 |   |      _____|____    
 S   OP    S     OP   S  
 |   |     |     |    |   
NUM ADD   NUM   MULT NUM 
 |   |     |     |    |   
two plus three times four



Each parse tree represents a structured arithmetic expression. Manually calculate the value of the resulting equation for each of the parse trees.

<!--
BEGIN QUESTION
name: parsed_equation_result
-->

In [7]:
#TODO
result_tree1 = (2+3)*4
result_tree2 = 2+(3*4)

In [8]:
grader.check("parsed_equation_result")

All tests passed!

We got two different parse trees for this simple expression. The occurrence of different structural interpretations of the same text is called _structural ambiguity_ or _syntactic ambiguity_. Since natural language is oftentimes ambiguous, this is a very real concern.

In this particular case, the two syntactic structures corresponded to two different semantic values. As an exercise, try to construct an ambiguous expression (name it `pseudo_ambiguous`) such that all of its parse trees correspond to the same value, thereby demonstrating that not all structural ambiguity leads to semantic ambiguity.

<!--
BEGIN QUESTION
name: redundant_parses
-->

In [9]:
# TODO - construct an ambiguous expression such that all of its parse
# trees correspond to the same value. `pseudo_ambiguous` should be
# a string.
pseudo_ambiguous = "two plus two times two"

In [10]:
grader.check("redundant_parses")

All tests passed!

One approach to dealing with the issue of syntactic ambiguity is by defining a scoring system to score the possible parses and choosing the highest scoring tree. We will see how this can be done by taking a probabilistic approach to CFG.

# Probabilistic context-free grammars

To assign probabilities to strings, we will use a probabilistic context-free grammar (PCFG), a CFG in which each rule is augmented with a probability. A PCFG rule will be notated
$$A \to \beta\ [p]$$
where $A$ is a nonterminal, $\beta$ is a sequence of terminals and nonterminals, and $p$ is a probability associated with the rule.

We'll write $\Prob(\beta \given A)$ for the probability associated with the rule $A \to \beta$.

To constitute a valid probability distribution we require that for every nonterminal $A$
$$\sum_{A \to \beta \in \cal{P}} \Prob(\beta \given A) = 1$$
where $\cal{P}$ is the set of CFG productions of the grammar. That is, the probabilities associated with all rules with the same left-hand side must sum to one.

Define `probabilistic_arithmetic_grammar` to be a probabilistic version of `arithmetic grammar` above, where the nonterminal probability distributions are **as uniform across the productions as possible**.

> You'll use the NLTK `nltk.PCFG.fromstring` function, which allows you to add the probabilities in brackets after each right-hand side, just as we've been doing above. For example, to notate `NUM -> 'zero'` as of probability 0.5, use `NUM -> 'zero' [0.5]`.

<!--
BEGIN QUESTION
name: uniform_probabilities
-->

In [11]:
# TODO - define `probabilistic_arithmetic_grammar`. Round to
#        *3* significant figures if not divisible.
probabilistic_arithmetic_grammar = nltk.PCFG.fromstring(f"""
    S -> NUM [0.5]| S OP S [0.5]
    OP -> ADD [0.5]| MULT [0.5]

    NUM -> 'zero' [{1/11}] | 'one' [{1/11}] | 'two' [{1/11}]
    NUM -> 'three' [{1/11}] | 'four' [{1/11}] | 'five' [{1/11}]
    NUM -> 'six' [{1/11}] | 'seven' [{1/11}] | 'eight' [{1/11}]
    NUM -> 'nine' [{1/11}] | 'ten' [{1/11}]

    ADD -> 'plus' [1]
    MULT -> 'times' [1]
""")

In [12]:
grader.check("uniform_probabilities")

All tests passed!

We can use the [nltk.CFG.productions()](https://www.nltk.org/api/nltk.html?highlight=production#nltk.grammar.CFG.productions) method to get a list of the PCFG's productions:

In [13]:
probabilistic_arithmetic_grammar.productions()

[S -> NUM [0.5],
 S -> S OP S [0.5],
 OP -> ADD [0.5],
 OP -> MULT [0.5],
 NUM -> 'zero' [0.0909091],
 NUM -> 'one' [0.0909091],
 NUM -> 'two' [0.0909091],
 NUM -> 'three' [0.0909091],
 NUM -> 'four' [0.0909091],
 NUM -> 'five' [0.0909091],
 NUM -> 'six' [0.0909091],
 NUM -> 'seven' [0.0909091],
 NUM -> 'eight' [0.0909091],
 NUM -> 'nine' [0.0909091],
 NUM -> 'ten' [0.0909091],
 ADD -> 'plus' [1.0],
 MULT -> 'times' [1.0]]

Each of the productions in the list is an instance of the [ProbabilisticProduction](https://www.nltk.org/api/nltk.html?highlight=production#nltk.grammar.ProbabilisticProduction) class. Each such instance is defined by three parameters: its left hand side (`lhs`), right-hand side (`rhs`), and rule probability (`prob`). These attributes can be accessed separately:

In [14]:
## Extract the second rule
pprod_example = probabilistic_arithmetic_grammar.productions()[1]

## Display its various components
print(f'For the production "{pprod_example}":\n'
      f'left hand side of the rule is {pprod_example.lhs()}\n'
      f'right hand side of the rule is {pprod_example.rhs()}\n'
      f'probability of the rule is {pprod_example.prob()}')

For the production "S -> S OP S [0.5]":
left hand side of the rule is S
right hand side of the rule is (S, OP, S)
probability of the rule is 0.5


For non-probabilistic grammars, the class of productions is [Production](https://www.nltk.org/api/nltk.html?highlight=production#nltk.grammar.Production), which doesn't have a probability attribute and is only defined by its lhs and rhs attributes:

In [15]:
print(f'PCFG production: {probabilistic_arithmetic_grammar.productions()[1]} \n'
      f'      vs.\n'
      f'CFG production:  {arithmetic_grammar.productions()[1]}')

PCFG production: S -> S OP S [0.5] 
      vs.
CFG production:  S -> S OP S


# Parse tree probabilities

To use a PCFG to select among parse trees, we need to be able to calculate the probability of a parse tree as specified by the PCFG. We take the probability of a parse tree to be simply the product of the probabilities of each constituent in the tree, the probability of the rule associated with the constituent.

You'll use the PCFG `probabilistic_arithmetic_grammar` to calculate the probability of each of the parse trees in `parses`, the list of trees that were parsed from the `example` sentence.

To do that, you'll need to get all the productions used in a parse tree (using the [productions](https://www.nltk.org/api/nltk.html?highlight=production#nltk.tree.Tree.productions) method), find their probabilities, and multiply them together.

First, we will create a dictionary from the PCFG, so that we can easily access the rule probabilities. Write a function which accepts a PCFG and returns a dictionary whose keys are the CFG (not PCFG) productions and values are the associated probabilities.

> To construct a CFG production from a PCFG production, you can use `nltk.grammar.Production(production.lhs(), production.rhs())`.

<!--
BEGIN QUESTION
name: pcfg_to_dict
-->

In [16]:
from nltk.parse.featurechart import Production
#TODO - returns a dictionary whose keys are `nltk.grammar.Production` objects
#       and whose values are the associated probabilities
def pcfg_to_dict(pcfg):
  res = dict()
  for prod_example in pcfg.productions():
    production = nltk.grammar.Production(prod_example.lhs(), prod_example.rhs())
    res[production] = prod_example.prob()
  return res

In [17]:
grader.check("pcfg_to_dict")

All tests passed!

We can use the function you wrote to convert `probabilistic_arithmetic_grammar` to a dictionary and inspect it to make sure it's working.

In [18]:
pprint(pcfg_to_dict(probabilistic_arithmetic_grammar))

{ADD -> 'plus': 1.0,
 MULT -> 'times': 1.0,
 NUM -> 'eight': 0.09090909090909091,
 NUM -> 'five': 0.09090909090909091,
 NUM -> 'four': 0.09090909090909091,
 NUM -> 'nine': 0.09090909090909091,
 NUM -> 'one': 0.09090909090909091,
 NUM -> 'seven': 0.09090909090909091,
 NUM -> 'six': 0.09090909090909091,
 NUM -> 'ten': 0.09090909090909091,
 NUM -> 'three': 0.09090909090909091,
 NUM -> 'two': 0.09090909090909091,
 NUM -> 'zero': 0.09090909090909091,
 OP -> ADD: 0.5,
 OP -> MULT: 0.5,
 S -> NUM: 0.5,
 S -> S OP S: 0.5}


Now for the payoff: Write a function that takes a parse tree and a PCFG and returns the probability of the parse tree according to the PCFG. The `pcfg_to_dict` function you just wrote is likely to come in handy.

> Note that we are asking for the probability (not the log probability). We **don't work in log space** in this lab for simplicity, but for parse trees of longer sentences (which you'll see in the project) you might have to work in the log space to avoid underflows.

<!--
BEGIN QUESTION
name: parsed_trees_probs
-->

In [19]:
# TODO: returns the probability of the parse tree.
# `tree.productions() might be useful for getting the
#  productions of a parse tree
def parse_probability(tree, pcfg):
  res = 1
  map = pcfg_to_dict(pcfg)
  for val in tree.productions():
    res *= map[val]
  return res


In [20]:
grader.check("parsed_trees_probs")

All tests passed!

We'll use it to calculate and print out the probability of each parse tree.

In [21]:
for i, tree in enumerate(parses):
    print(f'Probability of parse tree {i+1} is '
          f'{parse_probability(tree, probabilistic_arithmetic_grammar):1.2e}')
    tree.pretty_print()

Probability of parse tree 1 is 5.87e-06
           S             
      _____|__________    
     S           |    |  
  ___|_____      |    |   
 S   OP    S     OP   S  
 |   |     |     |    |   
NUM ADD   NUM   MULT NUM 
 |   |     |     |    |   
two plus three times four

Probability of parse tree 2 is 5.87e-06
           S             
  _________|_____         
 |   |           S       
 |   |      _____|____    
 S   OP    S     OP   S  
 |   |     |     |    |   
NUM ADD   NUM   MULT NUM 
 |   |     |     |    |   
two plus three times four



<!-- BEGIN QUESTION -->

**Question:** Which of the trees is the most probable parse? Explain why. If the two have the same probability, explain why that is the case instead, and describe how you might adjust the rule probabilities if possible so that they have different probabilities.

<!--
BEGIN QUESTION
name: open_response_ambiguity
manual: true
-->

_The parse trees have the same probability due to sharing identical production rules. To differentiate the probabilities, we can introduce a new asymmetric production rule, such as S -> S OP NUM, with a distinct probability compared to the existing rule S -> S OP S. This asymmetry allows us to assign different probabilities to parse trees based on the presence of a numerical value or another subtree as the second operand. By incorporating this modification, we can generate parse trees with varying probabilities, capturing different interpretations or preferences in the parsing process._

<!-- END QUESTION -->



# Lexicalizing the grammar

In order to allow parse probabilities to be more sensitive to contexts, it turns out to be useful to _lexicalize_ the grammar -- splitting (some of the) nonterminals based on what particular words they dominate. There are many techniques for performing this lexicalization. For this grammar, we'll split the `S` nonterminal based on the main operator that it dominates (if any). We'll thus have nonterminals `S_ADD`, `S_MULT`, and `S_NUM`. Thus, instead of a rule `S -> S OP S`, we'll have rules like:

```
S_ADD -> S_NUM ADD S_NUM
S_ADD -> S_NUM ADD S_ADD
S_ADD -> S_NUM ADD S_MULT
S_ADD -> S_ADD ADD S_NUM
```
and so forth. By splitting the nonterminals (and hence the productions) in this way, we can assign different probabilities to cases where, for instance, the primary operator on the left is a number, or addition, or multiplication.

Here is the lexicalized grammar:

In [22]:
lexicalized_arithmetic_grammar = nltk.CFG.fromstring(
    """
    S -> S_NUM | S_ADD | S_MULT

    S_NUM -> NUM

    S_ADD -> S_NUM ADD S_NUM
    S_ADD -> S_NUM ADD S_ADD
    S_ADD -> S_NUM ADD S_MULT
    S_ADD -> S_ADD ADD S_NUM
    S_ADD -> S_ADD ADD S_ADD
    S_ADD -> S_ADD ADD S_MULT
    S_ADD -> S_MULT ADD S_NUM
    S_ADD -> S_MULT ADD S_ADD
    S_ADD -> S_MULT ADD S_MULT

    S_MULT -> S_NUM MULT S_NUM
    S_MULT -> S_NUM MULT S_ADD
    S_MULT -> S_NUM MULT S_MULT
    S_MULT -> S_ADD MULT S_NUM
    S_MULT -> S_ADD MULT S_ADD
    S_MULT -> S_ADD MULT S_MULT
    S_MULT -> S_MULT MULT S_NUM
    S_MULT -> S_MULT MULT S_ADD
    S_MULT -> S_MULT MULT S_MULT

    NUM -> 'zero'   | 'one'    | 'two'
    NUM -> 'three'  | 'four'   | 'five'
    NUM -> 'six'    | 'seven'  | 'eight'
    NUM -> 'nine'   | 'ten'

    ADD -> 'plus'
    MULT -> 'times'
    """
)

<!-- BEGIN QUESTION -->

Use this grammar to parse the example phrase ("two plus three times four") defined as `example` above.

<!--
BEGIN QUESTION
name: lexicalized_parse
manual: true
-->

In [23]:
# TODO - parse `example` using the lexicalized grammar. `lexicalized_parses`
#        should be a list of parses.
lexicalized_parses = list(nltk.parse.BottomUpChartParser(lexicalized_arithmetic_grammar).parse(example.split()))

In [24]:
grader.check("lexicalized_parse")

All tests passed!

<!-- END QUESTION -->



Examine the trees, and make sure that you understand why they look the way they do. Notice that because of the lexicalization, the highest `S_` node corresponds to the highest operator in the parse -- `S_MULT` when `MULT` is the highest operator and `S_ADD` when `ADD` is the highest operator.

In [25]:
for i, tree in enumerate(lexicalized_parses):
  print(f"Possible parse {i+1}:\n")
  tree.pretty_print()

Possible parse 1:

              S               
              |                
            S_MULT            
         _____|____________    
      S_ADD          |     |  
   _____|_____       |     |   
S_NUM   |   S_NUM    |   S_NUM
  |     |     |      |     |   
 NUM   ADD   NUM    MULT  NUM 
  |     |     |      |     |   
 two   plus three  times  four

Possible parse 2:

             S               
             |                
           S_ADD             
   __________|_____           
  |    |         S_MULT      
  |    |      _____|______    
S_NUM  |   S_NUM   |    S_NUM
  |    |     |     |      |   
 NUM  ADD   NUM   MULT   NUM 
  |    |     |     |      |   
 two  plus three times   four



We can augment this grammar with probabilities as well.

Again, do so making the probabilities as uniform as possible.
<!--
BEGIN QUESTION
name: uniform_lexicalized_probabilities
-->

In [26]:
# TODO - define `probabilistic_lexicalized_arithmetic_grammar`.
#        Round to *3* significant figures if not divisible.
probabilistic_lexicalized_arithmetic_grammar = nltk.PCFG.fromstring(f"""
    S -> S_NUM [{1/3}] | S_ADD [{1/3}]| S_MULT [{1/3}]

    S_NUM -> NUM [1.0]

    S_ADD -> S_NUM ADD S_NUM [{1/9}]
    S_ADD -> S_NUM ADD S_ADD [{1/9}]
    S_ADD -> S_NUM ADD S_MULT [{1/9}]
    S_ADD -> S_ADD ADD S_NUM [{1/9}]
    S_ADD -> S_ADD ADD S_ADD [{1/9}]
    S_ADD -> S_ADD ADD S_MULT [{1/9}]
    S_ADD -> S_MULT ADD S_NUM [{1/9}]
    S_ADD -> S_MULT ADD S_ADD [{1/9}]
    S_ADD -> S_MULT ADD S_MULT [{1/9}]

    S_MULT -> S_NUM MULT S_NUM [{1/9}]
    S_MULT -> S_NUM MULT S_ADD [{1/9}]
    S_MULT -> S_NUM MULT S_MULT [{1/9}]
    S_MULT -> S_ADD MULT S_NUM [{1/9}]
    S_MULT -> S_ADD MULT S_ADD [{1/9}]
    S_MULT -> S_ADD MULT S_MULT [{1/9}]
    S_MULT -> S_MULT MULT S_NUM [{1/9}]
    S_MULT -> S_MULT MULT S_ADD [{1/9}]
    S_MULT -> S_MULT MULT S_MULT [{1/9}]

    NUM -> 'zero' [{1/11}] | 'one' [{1/11}] | 'two' [{1/11}]
    NUM -> 'three' [{1/11}] | 'four' [{1/11}] | 'five' [{1/11}]
    NUM -> 'six' [{1/11}] | 'seven' [{1/11}] | 'eight' [{1/11}]
    NUM -> 'nine' [{1/11}] | 'ten' [{1/11}]

    ADD -> 'plus' [1.0]
    MULT -> 'times' [1.0]
    """
)

In [27]:
grader.check("uniform_lexicalized_probabilities")

All tests passed!

Using this PCFG, we can calculate the probabilities associated with the two parses of the example phrase.

In [28]:
for i, tree in enumerate(lexicalized_parses):
    print(f'Probability of parsed tree {i+1} is '
          f'{parse_probability(tree, probabilistic_lexicalized_arithmetic_grammar):1.2e}')
    tree.pretty_print()

Probability of parsed tree 1 is 3.09e-06
              S               
              |                
            S_MULT            
         _____|____________    
      S_ADD          |     |  
   _____|_____       |     |   
S_NUM   |   S_NUM    |   S_NUM
  |     |     |      |     |   
 NUM   ADD   NUM    MULT  NUM 
  |     |     |      |     |   
 two   plus three  times  four

Probability of parsed tree 2 is 3.09e-06
             S               
             |                
           S_ADD             
   __________|_____           
  |    |         S_MULT      
  |    |      _____|______    
S_NUM  |   S_NUM   |    S_NUM
  |    |     |     |      |   
 NUM  ADD   NUM   MULT   NUM 
  |    |     |     |      |   
 two  plus three times   four



Make sure that you understand why the parse probabilities are the way they are.

# Estimating rule probabilities from a corpus

In the previous section, you received a CFG augmented with rule probabilities that were arbitrarily stipulated. But where should rule probabilities come from? One way to generate rule probabilites is to learn them from a training corpus.

In this section you will use a toy corpus of sentences parsed according to the lexicalized grammar to generate maximum likelihood estimates of rule probabilities by counting the number of occurrences of a rule used in the corpus.

In [29]:
## The raw corpus, before splitting into separate phrases
corpus_raw = """
    # seven
    (S (S_NUM (NUM seven)))
    # one plus two
    (S (S_ADD (S_NUM (NUM one)) (ADD plus) (S_NUM (NUM two))))
    # two times three
    (S (S_MULT (S_NUM (NUM two)) (MULT times) (S_NUM (NUM three))))
    # two plus six times one
    (S (S_ADD (S_NUM (NUM two)) (ADD plus) (S_MULT (S_NUM (NUM six)) (MULT times) (S_NUM (NUM one)))))
    # two plus five times one
    (S (S_MULT (S_ADD (S_NUM (NUM two)) (ADD plus) (S_NUM (NUM five))) (MULT times) (S_NUM (NUM one))))
    # eight plus three plus seven
    (S (S_ADD (S_ADD (S_NUM (NUM eight)) (ADD plus) (S_NUM (NUM three))) (ADD plus) (S_NUM (NUM seven))))
    # two plus three times four
    (S (S_ADD (S_NUM (NUM two)) (ADD plus) (S_MULT (S_NUM (NUM three)) (MULT times) (S_NUM (NUM four)))))
    # eight times four times two
    (S (S_MULT (S_MULT (S_NUM (NUM eight)) (MULT times) (S_NUM (NUM four))) (MULT times) (S_NUM (NUM two))))
    # five times two plus one
    (S (S_ADD (S_MULT (S_NUM (NUM five)) (MULT times) (S_NUM (NUM two))) (ADD plus) (S_NUM (NUM one))))
    # five plus one times four
    (S (S_ADD (S_NUM (NUM five)) (ADD plus) (S_MULT (S_NUM (NUM one)) (MULT times) (S_NUM (NUM four)))))
    # two times three plus four
    (S (S_ADD (S_MULT (S_NUM (NUM two)) (MULT times) (S_NUM (NUM three))) (ADD plus) (S_NUM (NUM four))))
    # ten plus two times three
    (S (S_ADD (S_NUM (NUM ten)) (ADD plus) (S_MULT (S_NUM (NUM two)) (MULT times) (S_NUM (NUM three)))))
    # four times three plus two times one
    (S (S_ADD (S_MULT (S_NUM (NUM four)) (MULT times) (S_NUM (NUM three))) (ADD plus) (S_MULT (S_NUM (NUM two)) (MULT times) (S_NUM (NUM one)))))
    # four plus three times two plus one
    (S (S_ADD (S_ADD (S_NUM (NUM four)) (ADD plus) (S_MULT (S_NUM (NUM three)) (MULT times) (S_NUM (NUM two)))) (ADD plus) (S_NUM (NUM one))))
"""

def corpus_from_string(raw):
  """Return a corpus as a list of sentences.

  The `raw` corpus is split at newlines, trimmed of whitespace,
  and comment lines and blank lines are eliminated.
  """
  return list(filter(lambda x: x != '' and x[0] != '#',
                     map(lambda sent: sent.strip(),
                         raw.split('\n'))))

## The processed corpus we'll use
corpus = corpus_from_string(corpus_raw)

Recall that for the rule probabilities to define a valid probability distibution, the following needs to hold
$$\sum_{A \to \beta \in G} \Prob(\beta \given A) = 1$$
where $G$ is the set of productions.

In order to get an estimate for each production probability, we can count the number of occurrences of the production, normalizing by the number of occurrences of all productions with the same left-hand side.

\begin{align}
\Prob(\beta \given A)
  &= \frac{\cnt{A \to \beta}}{\sum_{\beta'} \cnt{A \to \beta'}} \\
  &= \frac{\cnt{A \to \beta}}{\cnt{A}}
\end{align}

We will define three functions:

1. `rule_counter` - accepts a list of sentences and returns a dictionary of rule counts (where the key is the NLTK CFG production (defined by the lhs and rhs) and the value is the number of rule occurrences)
2. `lhs_counter` - accepts a list of sentences and returns a dictionary of lhs counts (where the key is the lhs nonterminal and the value is the count of that nonterminal's occurences as a lhs)
3. `rule_probs` - accepts a list of sentences and returns a dictionary of rule probabilities (where the key is the production and the value is the rule probability).

Implement these functions as specified above.

<!--
BEGIN QUESTION
name: probs_from_corpus
-->

In [38]:
#TODO
def rule_counter(sentence_list):
  res = defaultdict(int)
  for sentence in sentence_list:
    parse_tree = nltk.Tree.fromstring(sentence)
    for production in parse_tree.productions():
      res[nltk.grammar.Production(production.lhs(), production.rhs())] += 1
  return res

#TODO
def lhs_counter(sentence_list):
  res = defaultdict(int)
  for sentence in sentence_list:
    parse_tree = nltk.Tree.fromstring(sentence)
    for production in parse_tree.productions():
      res[production.lhs()] += 1
  return res

#TODO
def rule_probs(sentence_list):
  res = defaultdict(int)
  rules_count = rule_counter(sentence_list)
  lhs_count = lhs_counter(sentence_list)
  for nonterminal, nonterminal_count in rules_count.items():
    res[nonterminal] = nonterminal_count / lhs_count[nonterminal.lhs()]
  return res

In [39]:
grader.check("probs_from_corpus")

All tests passed!

Now we can use the `rules_prob` function you wrote to get the rule probabilities from our corpus:

In [40]:
probs_from_corpus = rule_probs(corpus)
pprint(probs_from_corpus)

defaultdict(<class 'int'>,
            {ADD -> 'plus': 1.0,
             MULT -> 'times': 1.0,
             NUM -> 'eight': 0.05,
             NUM -> 'five': 0.075,
             NUM -> 'four': 0.15,
             NUM -> 'one': 0.175,
             NUM -> 'seven': 0.05,
             NUM -> 'six': 0.025,
             NUM -> 'ten': 0.025,
             NUM -> 'three': 0.175,
             NUM -> 'two': 0.275,
             S -> S_ADD: 0.7142857142857143,
             S -> S_MULT: 0.21428571428571427,
             S -> S_NUM: 0.07142857142857142,
             S_ADD -> S_ADD ADD S_NUM: 0.15384615384615385,
             S_ADD -> S_MULT ADD S_MULT: 0.07692307692307693,
             S_ADD -> S_MULT ADD S_NUM: 0.15384615384615385,
             S_ADD -> S_NUM ADD S_MULT: 0.38461538461538464,
             S_ADD -> S_NUM ADD S_NUM: 0.23076923076923078,
             S_MULT -> S_ADD MULT S_NUM: 0.07692307692307693,
             S_MULT -> S_MULT MULT S_NUM: 0.07692307692307693,
             S_MULT -> S_NU

Observe that the probabilities of the two rules `S_ADD -> S_NUM ADD S_MULT` and `S_MULT -> S_ADD MULT S_NUM` are now different from each other. (They were both the same in the previous grammar, since you made the probabilities as uniform as possible.)

NLTK allows us to infer a probabilistic grammar from a parsed corpus like this one using [`nltk.induce_pcfg`](https://www.nltk.org/api/nltk.grammar.html#nltk.grammar.induce_pcfg). Let's do that.

In [41]:
def flatten(l):
    return sum(l, [])

def pcfg_from_trees(trees):
    return nltk.induce_pcfg(nltk.Nonterminal('S'),
                            flatten([nltk.Tree.fromstring(tree).productions()
                                     for tree in trees]))

induced_pcfg = pcfg_from_trees(corpus)

print(induced_pcfg)

Grammar with 23 productions (start state = S)
    S -> S_NUM [0.0714286]
    S_NUM -> NUM [1.0]
    NUM -> 'seven' [0.05]
    S -> S_ADD [0.714286]
    S_ADD -> S_NUM ADD S_NUM [0.230769]
    NUM -> 'one' [0.175]
    ADD -> 'plus' [1.0]
    NUM -> 'two' [0.275]
    S -> S_MULT [0.214286]
    S_MULT -> S_NUM MULT S_NUM [0.846154]
    MULT -> 'times' [1.0]
    NUM -> 'three' [0.175]
    S_ADD -> S_NUM ADD S_MULT [0.384615]
    NUM -> 'six' [0.025]
    S_MULT -> S_ADD MULT S_NUM [0.0769231]
    NUM -> 'five' [0.075]
    S_ADD -> S_ADD ADD S_NUM [0.153846]
    NUM -> 'eight' [0.05]
    NUM -> 'four' [0.15]
    S_MULT -> S_MULT MULT S_NUM [0.0769231]
    S_ADD -> S_MULT ADD S_NUM [0.153846]
    NUM -> 'ten' [0.025]
    S_ADD -> S_MULT ADD S_MULT [0.0769231]


We'll use NLTK's implementation of the probabilistic CKY algorithm ([`nltk.ViterbiParser`](https://www.nltk.org/api/nltk.parse.viterbi.html#nltk.parse.viterbi.ViterbiParser)) to generate the best parse for some strings according to this induced PCFG. (You'll implement this yourself in lab 3-4.)

In [42]:
induced_parser = nltk.ViterbiParser(induced_pcfg)

Use this parser to parse the `example` phrase "two plus three times four" from above. Which parse does it return? Do you understand why?

> Be careful. The parser returns a Python generator of the parses, not a list. You can't use the generator twice, so you should save the `induced_grammar_parses` as a list constructed from the generator object to pass all of the tests.

<!--
BEGIN QUESTION
name: induced_grammar_parses
-->

In [43]:
# TODO - parse `example` using `induced_parser`
induced_grammar_parses = list(induced_parser.parse(example.split()))

In [44]:
grader.check("induced_grammar_parses")

All tests passed!

In [45]:
for i, tree in enumerate(induced_grammar_parses):
    print(f'Probability of parse tree {i+1} is '
          f'{parse_probability(tree, induced_pcfg):1.2e}')
    tree.pretty_print()

Probability of parse tree 1 is 1.68e-03
             S               
             |                
           S_ADD             
   __________|_____           
  |    |         S_MULT      
  |    |      _____|______    
S_NUM  |   S_NUM   |    S_NUM
  |    |     |     |      |   
 NUM  ADD   NUM   MULT   NUM 
  |    |     |     |      |   
 two  plus three times   four



Now consider a new example:

In [46]:
example2 = "three plus nine plus two"

How many parses there are for this new expression "three plus nine plus two" according to the induced PCFG? Set the variable in the next cell accordingly.

<!--
BEGIN QUESTION
name: parse_count_2
-->

In [47]:
# TODO
example2_parse_count = 0

In [48]:
grader.check("parse_count_2")

All tests passed!

<!-- BEGIN QUESTION -->

**Question:** You undoubtedly obtained a number of parses for this second example that didn't seem appropriate. With a _single word_, what technique that you've learned would be appropriate to solve this problem.

<!--
BEGIN QUESTION
name: open_response_example_2
manual: true
-->

_smoothing._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Question:** The example that we provided of an ambiguity in arithmetic expressions is admittedly quite artificial. Can you think of other examples in natural language, more natural than the arithmetic expressions?

<!--
BEGIN QUESTION
name: open_response_other_examples
manual: true
-->

_Consider the sentence: 'I saw a man with a telescope.' This sentence exhibits an ambiguity. It can be interpreted in two ways: either the speaker saw a man while using a telescope, or the speaker saw a man who had a telescope. The phrase 'with a telescope' can modify either the action of seeing or the noun 'man'._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

# Lab debrief

**Question:** We're interested in any thoughts your group has about this lab so that we can improve this lab for later years, and to inform later labs for this year. Please list any issues that arose or comments you have to improve the lab. Useful things to comment on include the following:

* Was the lab too long or too short?
* Were the readings appropriate for the lab?
* Was it clear (at least after you completed the lab) what the points of the exercises were?
* Are there additions or changes you think would make the lab better?

<!--
BEGIN QUESTION
name: open_response_debrief
manual: true
-->

_It was all good._

<!-- END QUESTION -->



# End of Lab 3-3

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [49]:
grader.check_all()

induced_grammar_parses:

    All tests passed!
    

lexicalized_parse:

    All tests passed!
    

parse_count_2:

    All tests passed!
    

parsed_equation_result:

    All tests passed!
    

parsed_trees_probs:

    All tests passed!
    

pcfg_to_dict:

    All tests passed!
    

probs_from_corpus:

    All tests passed!
    

redundant_parses:

    All tests passed!
    

uniform_lexicalized_probabilities:

    All tests passed!
    

uniform_probabilities:

    All tests passed!